# Four Point Bending Test
This notebook performs a four point bending test consisting of two rollers that are pushed down vertically onto a rod with two vertical supports at its ends:
```
    |   |
    o   o
 ===========
 ^         ^
```
This test is popular because it induces a constant bending strain between the two rollers, but is somewhat challenging to simulate since the rollers and supports must be able to slide along the rod.

The parameters of this simulation were chosen to reproduce a physical lab test we performed for our [Barcelona Pavilion](http://julianpanetta.com/publication/xshell_pavilion/) and confirm that we can use such a bending test to accurately measure the rod's Young's modulus. We also compare against the known analytical solution for this test.

In [ ]:
import sys
sys.path.append('..')
import elastic_rods
import numpy as np
from typing import NamedTuple
from bending_validation import suppress_stdout as so

In [ ]:
rodWidth = 520
npts = 499
midpt = (npts + 1) // 2
thetaOffset = 3 * npts
# Contacts for support and loading.
# These will cause the vertex closest to xCoord to have its y coordinate
# constrained to the current displacement magnitude times "yDisplacementFactor"
class Contact(NamedTuple):
    xCoord: float
    yDisplacementFactor: float
contacts = [Contact(-212.5, 0), Contact(212.5, 0), Contact(-62.5, -1), Contact(62.5, -1)]

In [ ]:
pts = np.pad(np.linspace(-rodWidth / 2, rodWidth / 2, npts)[:,np.newaxis], [(0, 0), (0, 2)], mode='constant')
r = elastic_rods.ElasticRod(pts)
r.setMaterial(elastic_rods.RodMaterial('rectangle', 4.0e10 / 1e6, 0.3, [12, 8], stiffAxis=elastic_rods.StiffAxis.D2, keepCrossSectionMesh=True))

In [ ]:
import linkage_vis
view = linkage_vis.LinkageViewer(r)
view.show()

In [ ]:
rigidMotionVars  = [3 * midpt, 3 * midpt + 2] # pin x and z translation
rigidMotionVars += [2]                        # pin rotation around y axis (z comp. of arbitrary vtx)
rigidMotionVars += [thetaOffset]              # pin rotation around x axis

In [ ]:
def vtxAtXCoord(x):
    return np.argmin(np.abs(np.array(r.deformedPoints())[:, 0] - x))

def updateContacts(displacementMag):
    currDoFs = r.getDoFs()
    contactVars = []
    for contact in contacts:
        # Contact affects y component of the vertex
        var = 3 * vtxAtXCoord(contact.xCoord) + 1
        contactVars.append(var)
        currDoFs[var] = contact.yDisplacementFactor * displacementMag
    r.setDoFs(currDoFs)
    return contactVars

In [ ]:
import time
import py_newton_optimizer

maxDisplacementMag = 37.5
#maxDisplacementMag = 10

opts = py_newton_optimizer.NewtonOptimizerOptions()
opts.niter = 1000
opts.useIdentityMetric = False
opts.useNegativeCurvatureDirection = True
opts.gradTol = 1e-2
opts.verbose = 0
displacements = np.linspace(0, maxDisplacementMag, 50)
forces = []

for displacementMag in displacements[0:50]:
    # print(displacementMag)
    oldContactVars = []
    for i in range(10): # prevent cycling... (usually happens on border between rounding)
        contactVars = updateContacts(displacementMag)
        if (oldContactVars == contactVars): break
        # print("\t", contactVars)
        # time.sleep(0.05)
        oldContactVars = contactVars
        elastic_rods.compute_equilibrium(r, fixedVars=rigidMotionVars + contactVars, options=opts)
    forces.append(np.sum(r.gradient()[contactVars[0:2]]))
    # print(contactVars)
view.update(preserveExisting=False)

In [ ]:
contactVtxs = (np.array(updateContacts(displacementMag)) - 1) // 3

In [ ]:
r.gradient()[contactVars]

In [ ]:
np.array(r.deformedPoints())[contactVtxs]

In [ ]:
 from matplotlib import pyplot as plt
plt.plot(displacements, np.array(forces))
plt.xlabel('Displacement (mm)')
plt.ylabel('Force (N)')
plt.show()

In [ ]:
I = elastic_rods.RodMaterial('rectangle', 1, 0.3, [8, 12]).bendingStiffness.lambda_2

In [ ]:
a = 150 # length between left support and left load
L = 425 # full length between supports

In [ ]:
# Recover Young's modulus from the force/displacement data
# using the analytical formula for the deflection at the loads:
#      d = F * a^2 / (12 * E * I) (3L - 4a)
# where F is the total force applied by *both* rollers.
forces = np.array(forces)
(forces[1:] * a * a * (3 * L - 4 * a)) / (12 * I * displacements[1:])

## Validate stresses in the beam

In [ ]:
# Compare against the analytical solution for the stress in a rod under 4pt bending
F = forces[-1]
x = np.array(r.deformedPoints())[:, 0]
y = 8 / 2
analyticalStresses = np.zeros_like(x)
contact_x = (L / 2 - a)
for i in range(len(x)):
    stress = 0.0
    if (x[i] > -L / 2) and (x[i] < -contact_x):
        stress = F / 2 * (x[i] + L / 2) * y / I
    if (x[i] >= -contact_x) and (x[i] <= contact_x):
        stress = F / 2 * a * y / I
    if (x[i] > contact_x) and (x[i] < L / 2):
        stress = F / 2 * (L / 2 - x[i]) * y / I
    analyticalStresses[i] = stress

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(x, r.bendingStresses()[:, 0], label='DER simulation bending stress', linewidth=5)
plt.plot(x, r.maxStresses(elastic_rods.CrossSectionStressAnalysis.StressType.MaxPrincipal), label='DER simulation max principal stress', linewidth=5)
plt.plot(x, analyticalStresses, label='analytical formula')
plt.xlabel('x coordinate (mm)')
plt.ylabel('Max Stress (MPa)')
plt.legend()
plt.show()

In [ ]:
np.max(r.bendingStresses()),  np.max(analyticalStresses)

## Compute internal forces at rod interface

In [ ]:
gsm = elastic_rods.GradientStencilMaskCustom()

nv = r.numVertices()
ne = r.numEdges()
gsm.edgeStencilMask = np.zeros(r.numEdges())
mask = np.zeros(nv, dtype=bool)
mask[nv // 2 - 15:nv // 2 + 15] = True
gsm.vtxStencilMask = mask

In [ ]:
r.gradient(stencilMask=gsm)

In [ ]:
g = r.gradient(stencilMask=gsm)

In [ ]:
origDoF = r.getDoFs()

In [ ]:
perturbedDoFs = origDoF.copy()
perturbedDoFs[0:r.thetaOffset()] += 1 * np.random.uniform(low=-1, size=r.thetaOffset())
r.setDoFs(perturbedDoFs)
gperturb = r.gradient(stencilMask=gsm)

In [ ]:
view.update()